In [88]:
#! git pull

In [89]:
from azureml.core import Experiment, Run, Workspace
import azureml.core

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.14.0


In [90]:
import pandas as pd

ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'autoMLCensusRemoteONNX04'

experiment=Experiment(ws, experiment_name)

output = {}
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T


,
Subscription ID,6a0ec27b-1203-46fe-ade0-8c3003d5e4bc
Workspace,MLServiceWrokspace
Resource Group,MLServiceWrokspace
Location,northeurope
Experiment Name,autoMLCensusRemoteONNX04


In [91]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
#from sklearn.externals import joblib
import joblib
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import logging
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.explain.model._internal.explanation_client import ExplanationClient
from shap.datasets import adult

X, y = adult(display=True)
type(X)

pandas.core.frame.DataFrame

In [92]:
X['Loan'] = y
X.head(10)

,Age,Workclass,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Loan
0,39.00,State-gov,13.00,Never-married,Adm-clerical,Not-in-family,White,Male,2174.00,0.00,40.00,United-States,False
1,50.00,Self-emp-not-inc,13.00,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.00,0.00,13.00,United-States,False
2,38.00,Private,9.00,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.00,0.00,40.00,United-States,False
3,53.00,Private,7.00,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.00,0.00,40.00,United-States,False
4,28.00,Private,13.00,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.00,0.00,40.00,Cuba,False
5,37.00,Private,14.00,Married-civ-spouse,Exec-managerial,Wife,White,Female,0.00,0.00,40.00,United-States,False
6,49.00,Private,5.00,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0.00,0.00,16.00,Jamaica,False
7,52.00,Self-emp-not-inc,9.00,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.00,0.00,45.00,United-States,True
8,31.00,Private,14.00,Never-married,Prof-specialty,Not-in-family,White,Female,14084.00,0.00,50.00,United-States,True
9,42.00,Private,13.00,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178.00,0.00,40.00,United-States,True


In [93]:
train, test = train_test_split(X, test_size=0.2, random_state=42, shuffle=True)

In [94]:
print(train["Age"].count())
print(test["Age"].count())

26048
6513


In [95]:
#Train on a cluster

In [96]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

# choose a name for your cluster
AmlCompute_cluster_name = "automlcluster"
cluster_min_nodes = 0
cluster_max_nodes = 20
vm_size = "STANDARD_D2_V2"
autoscale_enabled = True


if AmlCompute_cluster_name in ws.compute_targets:
    compute_target = ws.compute_targets[AmlCompute_cluster_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + AmlCompute_cluster_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = cluster_min_nodes, 
                                                                max_nodes = cluster_max_nodes,
                                                                idle_seconds_before_scaledown=120)
    
    # create the cluster
    compute_target = ComputeTarget.create(ws, AmlCompute_cluster_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current BatchAI cluster status, use the 'status' property    
    print(compute_target.status.serialize())



found compute target. just use it. automlcluster


In [97]:
#Load the data into a datastore so the cluster can pick it up later
project_folder = './project'

if not os.path.isdir('dataCensus'):
    os.mkdir('dataCensus')
    
if not os.path.exists(project_folder):
    os.makedirs(project_folder)

pd.DataFrame(train).to_csv("dataCensus/X_trainCensus.csv", index=False)
pd.DataFrame(test).to_csv("dataCensus/y_trainCensus.csv", index=False)

ds = ws.get_default_datastore()
ds.upload(src_dir='./dataCensus', target_path='storesdataCensus', overwrite=True, show_progress=True)

Uploading an estimated of 2 files
Uploading ./dataCensus/X_trainCensus.csv
Uploaded ./dataCensus/X_trainCensus.csv, 1 files out of an estimated total of 2
Uploading ./dataCensus/y_trainCensus.csv
Uploaded ./dataCensus/y_trainCensus.csv, 2 files out of an estimated total of 2
Uploaded 2 files


$AZUREML_DATAREFERENCE_d88abfee192242fb9bc971dd49fe3ce4

In [184]:
#Get the pointers to the data
datastore = ws.get_default_datastore()
from azureml.core import Dataset
X = Dataset.Tabular.from_delimited_files(path = [(datastore, 'storesdataCensus/X_trainCensus.csv')])
y = Dataset.Tabular.from_delimited_files(path = [(datastore, 'storesdataCensus/y_trainCensus.csv')])

#X = dprep.auto_read_file(path=ds.path('storesdata/X_train.csv'))
#y = dprep.auto_read_file(path=ds.path('storesdata/y_train.csv'))

In [185]:
X.to_pandas_dataframe().head()

,Age,Workclass,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Loan
0,33.00,Local-gov,13.00,Never-married,Prof-specialty,Not-in-family,White,Female,0.00,0.00,50.00,United-States,True
1,36.00,Private,11.00,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.00,1887.00,50.00,United-States,True
2,58.00,Self-emp-not-inc,5.00,Separated,Craft-repair,Not-in-family,White,Male,0.00,0.00,40.00,United-States,False
3,21.00,Private,11.00,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.00,0.00,46.00,United-States,False
4,27.00,Private,10.00,Divorced,Other-service,Not-in-family,White,Male,0.00,0.00,40.00,United-States,False


In [100]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

# create a new runconfig object
run_config = RunConfiguration()

run_config.target = compute_target

# enable Docker 
run_config.environment.docker.enabled = True

# set Docker base image to the default CPU-based image
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE

# use conda_dependencies.yml to create a conda environment in the Docker image for execution
run_config.environment.python.user_managed_dependencies = False

conda = CondaDependencies.create(
#    pip_packages=['azureml-defaults', 'azureml-dataprep[fuse,pandas]', 'azureml-sdk'],
    pip_packages=['azureml-defaults', 'azureml-sdk'],
    pin_sdk_version=True
)

run_config.environment.python.conda_dependencies = conda


In [101]:
settings = {
    "debug_log": 'automl_census.txt'
}


AutoML_config = AutoMLConfig(task = 'classification',
                            primary_metric = 'accuracy',
                            run_configuration=run_config,
                            iterations = 20,
                            path = project_folder,
                            #n_cross_validations=4,
                            enable_voting_ensemble = False,
                            enable_stack_ensemble = False,
                            training_data = X,
                            label_column_name = "Loan",
                            enable_onnx_compatible_models=True,
                            #blacklist_models = ['model list'],
                            max_concurrent_iterations = 20,
                            **settings
                        )


In [102]:
#Run the automl
from azureml.core.experiment import Experiment
experiment=Experiment(ws, experiment_name)
run = experiment.submit(AutoML_config)
#local_run = experiment.submit(AutoML_config, show_output=True, )

Running on remote or ADB.


In [103]:
#View the results in a widget
from azureml.widgets import RunDetails
RunDetails(run).show()
#While this is running you can look at the compute in the portal to see the state of the nodes

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [106]:
run.wait_for_completion()

{'runId': 'AutoML_38a858e6-9d41-4f16-95e8-75a0af28f245',
 'target': 'automlcluster',
 'status': 'Completed',
 'startTimeUtc': '2020-11-17T17:12:13.19588Z',
 'endTimeUtc': '2020-11-17T17:27:27.475658Z',
 'properties': {'num_iterations': '20',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'automlcluster',
  'AMLSettingsJsonString': '{"path":null,"name":"autoMLCensusRemoteONNX04","subscription_id":"6a0ec27b-1203-46fe-ade0-8c3003d5e4bc","resource_group":"MLServiceWrokspace","workspace_name":"MLServiceWrokspace","region":"northeurope","compute_target":"automlcluster","spark_service":null,"azure_service":"remote","_local_managed_run_id":null,"many_models":false,"iterations":20,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":null,"y_max":null,"num_classes":null,"f

In [107]:
for e in Experiment.list(ws):
        if e.name == experiment_name:
            expr = e
print(expr)

Experiment(Name: autoMLCensusRemoteONNX04,
Workspace: MLServiceWrokspace)


In [108]:
#for r in expr.get_runs():
#    print(r.name)
#    if r.name == 'AutoML_25eb02e4-3933-44a0-a885-3fba532d54bc':
#        print('a')
#
##AutoML_25eb02e4-3933-44a0-a885-3fba532d54bc

In [109]:
from tqdm import tqdm
runs = {}
run_metrics = {}
runsorder = {}
i=0

for r in expr.get_runs():
    #if r.name == 'AutoML_25eb02e4-3933-44a0-a885-3fba532d54bc':
    metrics = r.get_metrics()
    for s in tqdm(r.get_children()):
        #print(s.name)
        metrics = s.get_metrics()
        if 'accuracy' in metrics.keys():
            i=i+1
            runs[s.id] = s
            run_metrics[s.id] = metrics

20it [00:17,  1.13it/s]


In [110]:
best_run_id = min(run_metrics, key = lambda k : run_metrics[k]['accuracy'])

In [111]:
bestrun = runs[best_run_id]
bestrun.get_file_names()

['accuracy_table',
 'automl_driver.py',
 'azureml-logs/55_azureml-execution-tvmps_2c48aba30b460aba05a20394d33a8306787732b691ad3d9823a56443809055ea_d.txt',
 'azureml-logs/65_job_prep-tvmps_2c48aba30b460aba05a20394d33a8306787732b691ad3d9823a56443809055ea_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_2c48aba30b460aba05a20394d33a8306787732b691ad3d9823a56443809055ea_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'confusion_matrix',
 'logs/azureml/101_azureml.log',
 'logs/azureml/azureml_automl.log',
 'logs/azureml/dataprep/python_span_67f21b62-48ec-4123-aa48-d9983e949bd2.jsonl',
 'logs/azureml/dataprep/python_span_934d5f15-235e-4575-8d26-8dc5d85782c5.jsonl',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/conda_env_v_1_0_0.yml',
 'outputs/env_dependencies.json',
 'outputs/model.onnx',
 'outputs/model.pkl',
 'outputs/model_onnx.json',
 'outputs/pipeline_graph.json',
 'outputs/scoring_file_v_1_0_0.py']

In [112]:
#Get the model filename
for f in bestrun.get_file_names():
    if(f[-5:] == '.onnx'):
        modelfilename = f
print(modelfilename)

outputs/model.onnx


In [179]:
#Download model
modelload = bestrun.register_model(model_name='model', model_path=modelfilename)
onnxfilename = modelload.download(exist_ok = True)

In [180]:
import onnx
onnx_model = onnx.load_model(onnxfilename)


In [187]:
import mlflow
import mlflow.onnx

from mlflow.models.signature import infer_signature

experiment_name = 'adultc_predict_exp'
artifact_path = 'adultc_predict_artifact'

mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())
mlflow.set_experiment(experiment_name)

with mlflow.start_run() as run:
    # Infer signature
    input_sample = X.to_pandas_dataframe().head(1)
    output_sample = pd.DataFrame(columns=['output_label'], data=[1])
    signature = infer_signature(input_sample, output_sample)

    # Save the model to the outputs directory for capture
    mlflow.onnx.log_model(onnx_model, artifact_path, signature=signature, input_example=input_sample)

    # Register the model to AML model registry
    mlflow.register_model('runs:/' + run.info.run_id + '/' + artifact_path, 'adultc_predict')

2020/11/17 18:01:37 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed, possibly due older server version. The model artifacts have been logged successfully under azureml://experiments/adultc_predict_exp/runs/ecfb7f39-8889-4d9b-8d1d-bf5152c9edb5/artifacts. In addition to exporting model artifacts, MLflow clients 1.7.0 and above attempt to record model metadata to the  tracking store. If logging to a mlflow server via REST, consider  upgrading the server version to MLflow 1.7.0 or above.
Registered model 'adultc_predict' already exists. Creating a new version of this model...
Created version '3' of model 'adultc_predict'.
